# Copy of AUC and histograms
* to try the rescoring
* uses weighed average

In [ ]:
# import libraries
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from spyrmsd import rmsd
import spyrmsd.molecule
import numpy as np
import prolif as plf
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
import math
import json
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from sklearn.model_selection import ParameterGrid

In [ ]:
# define list of bonds to find
# taken from rescore.ipynb file (optimalization of weights)

# bonds dicionary taken from optimalization using aligned bonds and -1.5 penalty for "extra" bonds
    # penalty now implemented also here
bonds = {'MET153.A_Hydrophobic': 0.84, 'MET156.A_VdWContact': 4.38647000624677, 'PHE368.A_Hydrophobic': 0.84, 'LYS105.A_VdWContact': 5.945898068663053, 'TYR155.A_Hydrophobic': 1.7772209283676828, 'LEU205.A_Hydrophobic': 3.6575552174288277, 'MET153.A_VdWContact': -1.64681891801749, 'PHE87.A_VdWContact': -0.9715120305947635, 'GLU154.A_VdWContact': 2.525721827497242, 'LYS105.A_PiCation': 8.28901702882621, 'ASP216.A_VdWContact': 0.37748685695342554, 'PHE87.A_Hydrophobic': 1.3372209283676828, 'VAL137.A_Hydrophobic': -1.8709829965975266, 'PHE368.A_VdWContact': -1.3194280624162824, 'GLY88.A_VdWContact': 1.2636909128609273, 'ALA215.A_VdWContact': 0.2, 'PHE120.A_Hydrophobic': -1.3994280624162825, 'PHE120.A_VdWContact': 2.4688780984537737, 'VAL90.A_Hydrophobic': -4.0099294855315035, 'VAL137.A_VdWContact': 0.16, 'PHE120.A_PiStacking': 0.16, 'ASP160.A_VdWContact': -2.450179005857511, 'ALA86.A_VdWContact': 0.8379453487529984, 'VAL90.A_VdWContact': 0.12, 'ASP202.A_VdWContact': -3.109598015506755, 'ASN203.A_VdWContact': -1.0942219755365032, 'GLU89.A_VdWContact': 0.08, 'ILE82.A_VdWContact': -2.490179005857511, 'ALA103.A_VdWContact': 3.7203398885308387, 'MET128.A_Hydrophobic': 0.04, 'ILE82.A_Hydrophobic': 0.04, 'LYS200.A_VdWContact': 2.693172001621254, 'SER116.A_VdWContact': 0.04, 'SER118.A_VdWContact': 0.04, 'LEU205.A_VdWContact': 0.04}

In [ ]:
# read protein molecule to calculate IFs on (save to PDBBlock)
protein_file = "../materials/2etr.pdb"
rdkit_prot = Chem.MolFromPDBFile(protein_file, removeHs=False)
protein = plf.Molecule(rdkit_prot)

In [ ]:
# calculate IFs of a molecule
# params: mol (rdkit.Molecule), protein (PDBBlock), bonds (dict)
# returns: score calculated from existence of certain bonds
def get_score(row, protein, bonds, docked_score, coef1, coef2, extra_bonds_penalty, not_found_penalty):
    bonds_found = row[row == True].index.tolist()
    common_bonds = set(bonds.keys()) & set(bonds_found)
    extra_bonds = set(bonds_found).difference(set(bonds.keys())) # bonds that are not "supposed" to be there (are in docked, not in aligned)
    not_found = set(bonds.keys()).difference(set(bonds_found)) # bonds that have a "score" but werent found


    common_sum = 0
    total_sum = 0
    for bond, num in bonds.items():
        total_sum = total_sum + num 
        if bond in common_bonds:
            common_sum = common_sum + num 
        elif bond in extra_bonds:
            common_sum = common_sum - extra_bonds_penalty
        elif bond in not_found:
            common_sum = common_sum - not_found_penalty

    score_temp = (common_sum / total_sum) * 10
    
    score = coef1*score_temp - coef2*docked_score 

    return score


In [ ]:

def get_auc(params):
    coef1, coef2, extra_bonds_penalty, not_found_penalty = params

    actives_scores = {}
    for x in range(24):
        df_actives = pd.read_csv(f"../materials/actives_decoys/ifs_glide/actives/actives_{x}.csv", index_col="idx")
        for idx, row in df_actives.iterrows():
            mol_id = row["mol_id"]
            docked_score = row["docked_score"]
            score = get_score(row, protein, bonds, docked_score,
                              coef1, coef2, extra_bonds_penalty, not_found_penalty)
            actives_scores[mol_id] = max(score, actives_scores.get(mol_id, -np.inf))

    decoy_scores = {}
    for x in range(93):
        df_decoys = pd.read_csv(f"../materials/actives_decoys/ifs_glide/decoys/decoys_{x}.csv", index_col="idx")
        for idx, row in df_decoys.iterrows():
            mol_id = row["mol_id"]
            docked_score = row["docked_score"]
            score = get_score(row, protein, bonds, docked_score,
                              coef1, coef2, extra_bonds_penalty, not_found_penalty)
            decoy_scores[mol_id] = max(score, decoy_scores.get(mol_id, -np.inf))

    a_scores = list(actives_scores.values())
    d_scores = list(decoy_scores.values())
    total_scores = a_scores + d_scores
    labels = [1]*len(a_scores) + [0]*len(d_scores)

    total_scores = [s for s in total_scores]
    auc = roc_auc_score(labels, total_scores)

    return auc  


In [ ]:
paramGrid = {
    "coef1": np.arange(0.08, 0.13, 0.01), #0.1
    "coef2": np.arange(0.88, 0.93, 0.01), #0.9
    "extra_bonds_penalty": [0.0],
    "not_found_penalty": [0.0],
    # "extra_bonds_penalty": np.arange(0.0, 5.0, 1.0),
    # "not_found_penalty": np.arange(1.0, 5.0, 1.0),
}
paramComb = ParameterGrid(paramGrid)

paramsAuc = []
combs_len = len(paramComb) 
print(f"num of combinations: {combs_len}")
combs_len = combs_len - 1
for i, comb in enumerate(paramComb):
    params = []
    for val in comb.values():
        params.append(val)
    auc =  get_auc(params)
    paramsAuc.append(auc)
    print(f"Hotovo: {i} / {combs_len}\n\tAUC = {auc}\n\tbestParams: {comb}")


bestParams = paramComb[np.argmax(paramsAuc)]
print(f"Best params: {bestParams}")
print(f"With AUC = {max(paramsAuc)}")



### Best Params Graphs

In [ ]:
# actives
#coef1, coef2, extra_bonds_penalty, not_found_penalty

actives_scores = {}
for x in range(24):
    df_actives = pd.read_csv(f"../materials/actives_decoys/ifs_glide/actives/actives_{x}.csv", index_col="idx")
    for idx, row in df_actives.iterrows():
        mol_id = row["mol_id"]
        docked_score = row["docked_score"]
        score = get_score(row, protein, bonds, docked_score, bestParams["coef1"], bestParams["coef2"], bestParams["extra_bonds_penalty"], bestParams["not_found_penalty"])
        if mol_id in actives_scores:
            if actives_scores[mol_id] < score:
                actives_scores[mol_id] = score
        else:
            actives_scores[mol_id] = score

print(actives_scores)

In [ ]:
# decoys
decoy_scores = {}
for x in range(93):
    df_decoys = pd.read_csv(f"../materials/actives_decoys/ifs_glide/decoys/decoys_{x}.csv", index_col="idx")
    for idx, row in df_decoys.iterrows():
        mol_id = row["mol_id"]
        docked_score = row["docked_score"]
        score = get_score(row, protein, bonds, docked_score, bestParams["coef1"], bestParams["coef2"], bestParams["extra_bonds_penalty"], bestParams["not_found_penalty"])
        if mol_id in decoy_scores:
            if decoy_scores[mol_id] < score:
                decoy_scores[mol_id] = score
        else:
            decoy_scores[mol_id] = score

print(decoy_scores)

In [ ]:
a_scores = [float(score[1]) for score in actives_scores.items()]
print(a_scores)
d_scores = [float(score[1]) for score in decoy_scores.items()]

In [ ]:
from matplotlib import pyplot as plt
plt.hist(a_scores,density=True,bins=50,alpha=0.5,label="actives")
plt.hist(d_scores,density=True,bins=50,alpha=0.5,label="decoys")


plt.title("docking scores actives vs decoys")
plt.title("docking scores actives")
plt.xlabel("docking score")
plt.ylabel("density")
plt.legend()
plt.show()

In [ ]:
total_scores = a_scores+d_scores
labels = [1]*len(a_scores)+[0]*len(d_scores)

total_scores = [t for t in total_scores]  # auc needs negative values (new scores are positive)

auc = roc_auc_score(labels,total_scores)
fpr,tpr,_ = roc_curve(labels,total_scores)
plt.plot(fpr,tpr,label=f"AUC = {auc}")
plt.plot([i for i in range(2)],[i for i in range(2)], label="random")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("GLIDE SP ROC curve")
plt.legend()
plt.show()